importing packages

In [99]:
!pip install langchain_core langchain_chroma langchain_text_splitters langchain langchain_google_genai sentence_transformers langsmith

I0000 00:00:1724530065.799046      36 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [3]:
import pandas as pd
import numpy as np
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.schema import Document
from sentence_transformers import SentenceTransformer
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.documents import Document
from kaggle_secrets import UserSecretsClient


/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [106]:
user_secrets = UserSecretsClient()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = user_secrets.get_secret("LANGCHAIN_KEY")

In [4]:
data_directory='/kaggle/input/sanad-dataset'
data=[]
for folder in os.listdir(data_directory):
    for file in os.listdir(f'{data_directory}/{folder}'):

            file = open(f"{data_directory}/{folder}/{file}", "r",encoding="utf8")
            content = file.read()

            data.append({'category': folder, 'Content': content})

            file.close()           

In [7]:
data[0]

{'category': 'Finance',
 'Content': 'دبي - "الخليج":حصدت شعاع كابيتال جائزة "أفضل شركة لإدارة الأصول في الإمارات" خلال حفل توزيع جوائز الأعمال المصرفية في الشرق الأوسط لعام 2014 الذي تنظمه مجلة إيميا فايننس . وتم اختيار شعاع لنيل الجائزة للسنة الخامسة على التوالي، تقديراً للأداء القوي الذي يحققه فريقها لإدارة الأصول .وقال الشيخ مكتوم بن حشر آل مكتوم، رئيس مجلس الإدارة التنفيذي لشعاع كابيتال: "نحن فخورون بالتكريم الذي حصل عليه فريقنا لإدارة الأصول مرّة جديدة، وذلك تقديراً لتميّزه في تحقيق مستوى متسق من العائدات لصالح العملاء . ويعدّ الفوز بهذه الجائزة لخمس مرّات متتالية إنجازاً كبيراً، ودافعاً قوياً لفريق إدارة الأصول ليواصلوا تحقيق هذا الأداء المتميّز" .وبلغ العائد الذي حققه الصندوقان الرئيسيان لدى شعاع لإدارة الأصول، وهما صندوق البوابة العربية وصندوق بوابة الإمارات، 2 .16% و2 .30% على التوالي خلال الفترة الماضية من العام 2014 . وتمتلك صناديق شعاع، التي أطلقت في العام ،1999 أطول سجل استثماري في المنطقة وقد حصل فريق إدارة الأصول لدى الشركة على مجموعة كبيرة من الجوائز خلال السنوات الأخير

In [13]:
documents = [Document(page_content=text["Content"]) for text in data[:40000]]

splitting data into documents

In [14]:
len(documents)

40000

In [ ]:
documents

Embedding to find similarity between texts and find relevant 

In [18]:
class embedding:
    def __init__(self):
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
    def embed_documents(self,docs):
        embeddings= self.model.encode(docs)
        return embeddings.tolist()
    def embed_query(self,query):
        return self.model.encode(query).tolist()

In [19]:
embed_model=embedding()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [38]:
result=embed_model.embed_query("ما الجائزة التي حصدتها شعاع كابيتال ؟")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [39]:
len(result)

384

In [40]:
vector_database=Chroma.from_documents(documents=documents,embedding=embed_model)

KeyboardInterrupt: 

In [120]:
retriever=vector_database.as_retriever(search_type="similarity",search_kwargs={'k':3})

In [93]:
retrieved_docs=retriever.invoke("لماذا تأجل اجتماع الجمعية العمومية غير العادية ل الدار العقارية ؟")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [94]:
retrieved_docs[0].page_content

'تم تأجيل اجتماع الجمعية العمومية غير العادية لشركة الخليج للملاحة القابضة لعدم اكتمال النصاب القانوني، وسوف يعقد الاجتماع الثاني يوم الخميس 5/12/2013 الساعة الخامسة .'

In [95]:
retrieved_docs[1].page_content

'شهدت تداولات المطلعين بيع 52،8 ألف سهم تمويل، بأسعار تراوحت بين 0،927 و0،935 درهم .'

In [96]:
retrieved_docs[2].page_content

'تراجعت أرباح "الاتحاد العقارية" بنسبة 46% إلى 6 .858 مليون درهم في 2014 مقارنة مع 57 .1 مليار درهم في ،2013 كما تراجعت الإيرادات 56% إلى 06 .2 مليار درهم مقارنة مع 66 .4 مليار درهم .'

construct template

In [126]:
template="""
you are an AI powered QA Assistant to provide accurate, contextually relevant answers to customer questions.
at the end of the answer you have to thank the user. 
the answer in arabic .
you should answer based on the context provided:
{context}

what to do if answer is not included in the prompt for context:

1.you should appoligize to the user and say that you dont have the answer in your informations

for answer:
1. the output must be the answer only without any additional thoughts

Question: {question}

answer:
"""

In [127]:
custom_rag_prompt=PromptTemplate.from_template(template)

In [128]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [129]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
google_api_key = user_secrets.get_secret("GOOGLE_AI_STUDIO2")
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash",google_api_key=google_api_key,temperature=0)

In [130]:
rag_chain =(
    {"context":retriever | format_docs , 'question':RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()

)

In [133]:
answer=rag_chain.invoke("لماذا تأجل اجتماع الجمعية العمومية غير العادية ؟")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [134]:
print(answer)

تم تأجيل اجتماع الجمعية العمومية غير العادية لشركة الخليج للملاحة القابضة لعدم اكتمال النصاب القانوني. 

